# Crime data in Chicago (January 1, 2010 to December 30, 2019)

In [19]:
import pandas as pd
crime_data = pd.read_csv('crime_data.csv')
crime_data.dtypes

/Users/rachaelhutson/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


ID                        int64
Case Number              object
Date                     object
Block                    object
IUCR                     object
Primary Type             object
Description              object
Location Description     object
Arrest                     bool
Domestic                   bool
Beat                      int64
District                float64
Ward                    float64
Community Area          float64
FBI Code                 object
X Coordinate            float64
Y Coordinate            float64
Year                      int64
Updated On               object
Latitude                float64
Longitude               float64
Location                 object
dtype: object

In [20]:
import datetime

cols_to_remove = list(crime_data)

# Separating the date column into month, day, year, and time columns
Months = []
Days   = []
Years  = []
Hours  = []

for row in crime_data['Date']:
    d = datetime.datetime.strptime(row, '%m/%d/%Y %I:%M:%S %p')
    Months.append(d.month)
    Days.append(d.day)
    Years.append(d.year)
    Hours.append(d.hour)
    
crime_data['Month'] = Months
crime_data['Day'] = Days
crime_data['Year1'] = Years
crime_data['Hour'] = Hours

crime_data[-4:]

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Y Coordinate,Year,Updated On,Latitude,Longitude,Location,Month,Day,Year1,Hour
7234248,24560,JC279072,05/26/2019 06:48:00 AM,013XX W HASTINGS ST,0110,HOMICIDE,FIRST DEGREE MURDER,CHA PARKING LOT,True,False,...,1893853.0,2019,06/20/2020 03:48:45 PM,41.864278,-87.659660,"(41.864278228, -87.659660218)",5,26,2019,6
7234249,11707734,JC287730,07/01/2014 07:30:00 AM,063XX S NORMAL BLVD,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,NaN,False,False,...,NaN,2014,06/02/2019 04:09:42 PM,NaN,NaN,NaN,7,1,2014,7
7234250,11707239,JC287563,11/30/2017 09:00:00 AM,022XX S KOSTNER AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,False,False,...,NaN,2017,06/02/2019 04:09:42 PM,NaN,NaN,NaN,11,30,2017,9
7234251,24559,JC278908,05/26/2019 02:11:00 AM,013XX W HASTINGS ST,0110,HOMICIDE,FIRST DEGREE MURDER,STREET,False,False,...,1893853.0,2019,06/20/2020 03:48:45 PM,41.864278,-87.659682,"(41.864278357, -87.659682244)",5,26,2019,2


In [1]:
# 1-hot encoding for: Arrest, Location Description, Domestic, District, Primary Type, Community Area
discrete_variables = "Arrest, Location Description, Domestic, District, Primary Type, Community Area".split(", ")
print(discrete_variables)
for variable in discrete_variables:
    for dtype in crime_data[variable].dropna().unique():
        if (dtype != "nan"):
            crime_data[variable+"_"+str(dtype)]  = 1*(crime_data[variable] == dtype)

print(cols_to_remove)            
crime_data = crime_data.drop(columns=cols_to_remove)
crime_data = crime_data.drop(columns=['Arrest_False','Domestic_False'])
crime_data[-4:]


['Arrest', 'Location Description', 'Domestic', 'District', 'Primary Type', 'Community Area']


NameError: name 'crime_data' is not defined

In [ ]:

arr_true = crime_data[crime_data['Arrest_True'] == 1]
ax = arr_true['Month'].plot.hist(bins=12)
crime_data.std()

In [ ]:
#scale all variables to be between 0 and 1
#iterate through all columns
for c in crime_data.columns:
    print(c)
    if c != 'Year1':
        crime_data[c] = (crime_data[c]-crime_data[c].min())/(crime_data[c].max() - crime_data[c].min())

crime_data.std().sort_values()

In [ ]:
import numpy as np

X = crime_data.to_numpy()

In [ ]:
# run PCA analysis (this can probably be deleted)
import sklearn
from sklearn.decomposition import PCA

pca = PCA(n_components=5) #find 2 principal components
fit = pca.fit(X)
print(("Explained Variance: %s") % (fit.explained_variance_ratio_))

pc1, pc2, pc3, pc4, p5 = pca.components_

weights1 = list(zip(pc1, crime_data.columns))
weights1.sort()

print('Principal Component 1: \n (Negative)',weights1[:5], '\n\n (Positive)' ,weights1[-5:])

In [ ]:
# evaluate different trends
from sklearn.model_selection import train_test_split

print(crime_data.groupby(['Arrest_True'])['Arrest_True'].count())
print("\n")
print(crime_data.groupby(['Month'])['Arrest_True'].mean())
print("\n")
print(crime_data.groupby(['Month'])['Month'].count())
print("\n")
print(crime_data.groupby(['Hour'])['Arrest_True'].mean())
print("\n")
print(crime_data.groupby(['Hour'])['Hour'].count())


print(crime_data['Hour'].corr(crime_data['Arrest_True'], method='pearson'))
print(crime_data['Month'].corr(crime_data['Arrest_True'], method='pearson'))
print(crime_data['Day'].corr(crime_data['Arrest_True'], method='pearson'))

In [ ]:
# logistic regression model 

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import log_loss

def logmodel(X1, y, year):
    X1.drop(columns = ['Year1'])
    X = X1.to_numpy() #defines the features
    Y = y.to_numpy() #labels (or what we are predicting)

    #create a testing dataset
    X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,random_state=1)

    log = LogisticRegression(max_iter=210000) #model that we use
    model = log.fit(X_train,Y_train)
    
    Y_pred = log.predict(X_test) #evaluation and prediction
    
    # print year
    print(str(year)+"\n")
    
    # print model fitting information
    print(classification_report(Y_test, Y_pred))
    
    # print sorted weights corresponding to each feature
    l = log.coef_
    weights = list(zip(l[0], X1.columns))
    weights.sort()
    print(weights)
    
    
M = crime_data

# run model on each year
i = 2020
while i>2000:
    X = M[M['Year1'] == i].drop(columns = ['Arrest_True'])
    y = M[M['Year1'] == i]['Arrest_True']
    logmodel(X,y,i)
    i = i-1


In [ ]:
# more examples from class below ...

In [ ]:
%matplotlib inline  

import matplotlib.pyplot as plt

Y = pca.transform(X)
plt.scatter(Y[:,0], Y[:,1], alpha=0.1)
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.show()

Let's try to interpret this figure a bit more. Let's see where all the "Baathist" Arab countries lie on this plot:

In [ ]:
arab = ['Syria', 'Jordan', 'Kuwait', 'UAE']
arab_index = [list(flag.index.values).index(a) for a in arab]
plt.scatter(Y[:,0], Y[:,1])
plt.scatter(Y[arab_index,0], Y[arab_index,1],color='r')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.show()

Or we can look at all of the countries in northern europe

In [ ]:
ne = ['Norway', 'Denmark', 'Finland', 'Iceland']
ne_index = [list(flag.index.values).index(a) for a in ne]
plt.scatter(Y[:,0], Y[:,1])
plt.scatter(Y[ne_index,0], Y[ne_index,1],color='r')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.show()

## Clustering
When we were playing around with the data we manually used data clusters that we roughly knew about to validate our model. How do we do this automatically? Clustering is the task of dividing the population or data points into a number of groups such that data points in the same groups are more similar to other data points in the same group than those in other groups. In particular, we will focus on a simple form of clustering called "k-means". k-means clustering aims to partition n observations into k clusters in which each observation belongs to the cluster with the nearest mean, serving as a prototype of the cluster. 

Again, the mathematical details are better left for a machine learning class but you can see how to use scikit-learn to do this:

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters = 15) #break the data into 15 clusters
kmeans.fit(X)

In [ ]:
flag[kmeans.labels_ == 9]

In [ ]:
flag[kmeans.labels_ == 6]